In [ ]:


!nvidia-smi

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))
# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="YeynZkK8Y71dBWtp7oX7")
project = rf.workspace("shoeb-bashar-4m364").project("palletised_box")
version = project.version(1)
dataset = version.download("coco")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# if your dataset is in COCO format, this cell can be replaced by the following three lines:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "/content/palletised_box-1/train/_annotations.coco.json", "/content/palletised_box-1/train")
register_coco_instances("my_dataset_val", {}, "/content/palletised_box-1/valid/_annotations.coco.json", "/content/palletised_box-1/valid")
register_coco_instances("my_dataset_test", {}, "/content/palletised_box-1/test/_annotations.coco.json", "/content/palletised_box-1/test")

In [ ]:
my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train")
dataset_dicts = DatasetCatalog.get("my_dataset_train")

import random
from detectron2.utils.visualizer import Visualizer

for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=my_dataset_train_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
#We are importing our own Trainer Module here to use the COCO validation evaluation during training. Otherwise no validation eval occurs.

from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [ ]:
#new for batch size 12
#from .detectron2.tools.train_net import Trainer
#from detectron2.engine import DefaultTrainer
# select from modelzoo here: https://github.com/facebookresearch/detectron2/blob/master/MODEL_ZOO.md#coco-object-detection-baselines

from detectron2.config import get_cfg
#from detectron2.evaluation.coco_evaluation import COCOEvaluator
import os

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)


cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 12
cfg.SOLVER.BASE_LR = 0.01

cfg.SOLVER.WARMUP_ITERS = 50
cfg.SOLVER.MAX_ITER = 1492  # Adjusted to 1492 iterations for 100 epochs
cfg.SOLVER.STEPS = (746, 1119)  # Adjusted for 50% and 75% of total iterations
cfg.SOLVER.GAMMA = 0.005


cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2 #your number of classes + 1

cfg.TEST.EVAL_PERIOD = 150


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
import os

# List the contents of the ./output/ directory
output_files = os.listdir('./output/')
print(output_files)


In [ ]:
#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "my_dataset_test")
inference_on_dataset(trainer.model, val_loader, evaluator)

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from sklearn.metrics import precision_recall_curve, confusion_matrix
import numpy as np

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

cfg.DATASETS.TEST = ("my_dataset_test", )

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

predictor = DefaultPredictor(cfg)

test_metadata = MetadataCatalog.get("my_dataset_test")

test_loader = build_detection_test_loader(cfg, "my_dataset_test")

evaluator = COCOEvaluator("my_dataset_test", cfg, False, output_dir="./output/")
predictions = inference_on_dataset(predictor.model, test_loader, evaluator)


true_labels = []
predicted_labels = []

for prediction in predictions:
    instances = prediction["instances"]
    gt_boxes = instances.gt_boxes.tensor.numpy() if "gt_boxes" in instances.fields() else None
    scores = instances.scores.numpy()


    if gt_boxes is not None:
        true_labels.extend([1] * len(gt_boxes))

    predicted_labels.extend([1 if score > cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST else 0 for score in scores])

true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

In [ ]:
# Look at AP, mAP and other evaluation curves curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
import os

# List the contents of the ./output/ directory
output_files = os.listdir('./output/')
print(output_files)

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("my_dataset_test", )
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
predictor = DefaultPredictor(cfg)
test_metadata = MetadataCatalog.get("my_dataset_test")

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("my_dataset_test", )
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
predictor = DefaultPredictor(cfg)
test_metadata = MetadataCatalog.get("my_dataset_test")

from detectron2.utils.visualizer import ColorMode
import glob

for imageName in glob.glob('/content/palletised_box-1/test/*jpg'):
  im = cv2.imread(imageName)
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1],
                metadata=test_metadata,
                scale=0.8
                 )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])


In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("my_dataset_test", )
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.25   # set the testing threshold for this model
predictor = DefaultPredictor(cfg)
test_metadata = MetadataCatalog.get("my_dataset_test")

from detectron2.utils.visualizer import ColorMode
import glob

for imageName in glob.glob('/content/palletised_box-1/test/*jpg'):
  im = cv2.imread(imageName)
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1],
                metadata=test_metadata,
                scale=0.8
                 )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])


In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("my_dataset_test", )
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75   # set the testing threshold for this model
predictor = DefaultPredictor(cfg)
test_metadata = MetadataCatalog.get("my_dataset_test")

from detectron2.utils.visualizer import ColorMode
import glob

for imageName in glob.glob('/content/palletised_box-1/test/*jpg'):
  im = cv2.imread(imageName)
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1],
                metadata=test_metadata,
                scale=0.8
                 )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])


In [ ]:

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("my_dataset_test", )
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.50   # set the testing threshold for this model
predictor = DefaultPredictor(cfg)
test_metadata = MetadataCatalog.get("my_dataset_test")

from detectron2.utils.visualizer import ColorMode
import glob

for imageName in glob.glob('/content/palletised_box-1/test/*jpg'):
  im = cv2.imread(imageName)
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1],
                metadata=test_metadata,
                scale=0.8
                 )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])


In [1]:
import cv2
import glob
from google.colab.patches import cv2_imshow


for imageName in glob.glob('/content/palletised_box-1/test/*.jpg'):

    im = cv2.imread(imageName)


    outputs = predictor(im)


    instances = outputs["instances"].to("cpu")


    for i in range(len(instances)):

        bbox = instances.pred_boxes.tensor.numpy()[i]


        x1, y1, x2, y2 = bbox
        width = int(x2 - x1)
        length = int(y2 - y1)


        cv2.rectangle(im, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)


        print(f"Image: {imageName}, Instance {i + 1}: W = {width}mm, L = {length}mm")


        cv2.putText(im, f"W: {width}mm, L: {length}mm", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


    cv2_imshow(im)


    import cv2
import glob
from google.colab.patches import cv2_imshow


for imageName in glob.glob('/content/parcel-2/test/*.jpg'):

    im = cv2.imread(imageName)


    outputs = predictor(im)


    instances = outputs["instances"].to("cpu")


    for i in range(len(instances)):
        # Get the bounding box coordinates of the instance
        bbox = instances.pred_boxes.tensor.numpy()[i]


        x1, y1, x2, y2 = bbox
        width = int(x2 - x1)
        length = int(y2 - y1)


        if width * length > 300:
            box_size = "Large Box"
        elif width * length > 150:
            box_size = "Medium Box"
        else:
            box_size = "Small Box"

        # Overlay bounding box on the image
        cv2.rectangle(im, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

        # Print the box size on the image
        cv2.putText(im, box_size, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the image with bounding boxes and box sizes
    cv2_imshow(im)



In [ ]:
import cv2
import glob
from google.colab.patches import cv2_imshow


for imageName in glob.glob('/content/palletised_box-1/test/*.jpg'):

    im = cv2.imread(imageName)


    outputs = predictor(im)


    instances = outputs["instances"].to("cpu")


    for i in range(len(instances)):
        # Get the bounding box coordinates of the instance
        bbox = instances.pred_boxes.tensor.numpy()[i]


        x1, y1, x2, y2 = bbox
        width = int(x2 - x1)
        length = int(y2 - y1)


        cv2.rectangle(im, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)


        print(f"Image: {imageName}, Instance {i + 1}: W = {width}mm, L = {length}mm")


        cv2.putText(im, f"W: {width}mm, L: {length}mm", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


    cv2_imshow(im)


    import cv2
import glob
from google.colab.patches import cv2_imshow


for imageName in glob.glob('/content/parcel-2/test/*.jpg'):

    im = cv2.imread(imageName)


    outputs = predictor(im)


    instances = outputs["instances"].to("cpu")


    for i in range(len(instances)):

        bbox = instances.pred_boxes.tensor.numpy()[i]

        # Calculating width and height of the bounding box
        x1, y1, x2, y2 = bbox
        width = int(x2 - x1)
        length = int(y2 - y1)

        if width * length > 300:
            box_size = "Large Box"
        elif width * length > 150:
            box_size = "Medium Box"
        else:
            box_size = "Small Box"

        cv2.rectangle(im, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)


        cv2.putText(im, box_size, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


    cv2_imshow(im)



In [ ]:
import cv2
import glob
from google.colab.patches import cv2_imshow


for imageName in glob.glob('/content/palletised_box-1/test/*.jpg'):

    im = cv2.imread(imageName)


    outputs = predictor(im)


    instances = outputs["instances"].to("cpu")


    for i in range(len(instances)):
        # Get the bounding box coordinates of the instance
        bbox = instances.pred_boxes.tensor.numpy()[i]

        # Calculating width and height of the bounding box
        x1, y1, x2, y2 = bbox
        width = int(x2 - x1)
        length = int(y2 - y1)

        # Calculating the area of the bounding box
        area = width * length

        # Determining the box size based on the area as an extension of the fyp
        if area > 300:
            width = "Large Box"
        elif area > 150:
            width = "Medium Box"
        else:
            box_size = "Small Box"


        cv2.rectangle(im, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)


        cv2.putText(im, box_size, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


    cv2_imshow(im)


In [ ]:
import cv2
import glob
from google.colab.patches import cv2_imshow

#with pixel to mm scale factor
#Assuming a scale factor of 0.5 just to show an example for FYP (1 pixel = 0.5 millimeters)
pixel_to_mm_scale_factor = 0.5


for imageName in glob.glob('/content/palletised_box-1/test/*.jpg'):

    im = cv2.imread(imageName)


    outputs = predictor(im)


    instances = outputs["instances"].to("cpu")


    for i in range(len(instances)):

        bbox = instances.pred_boxes.tensor.numpy()[i]


        x1, y1, x2, y2 = bbox
        width_px = int(x2 - x1)
        length_px = int(y2 - y1)

        # Convert pixel dimensions to millimeters using the scale factor
        width_mm = width_px * pixel_to_mm_scale_factor
        length_mm = length_px * pixel_to_mm_scale_factor


        cv2.rectangle(im, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)


        print(f"Image: {imageName}, Instance {i + 1}: W = {width_mm:.2f}mm, L = {length_mm:.2f}mm")


        cv2.putText(im, f"W: {width_mm:.2f}mm, L: {length_mm:.2f}mm", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


    cv2_imshow(im)
